# Available Credit Limit Calculation - Examples / Tính Toán Hạn Mức Khả Dụng - Ví Dụ Minh Họa

This notebook demonstrates how to calculate available credit limits for bank credit with:
- **Hierarchical limits** (master/child/sub-child)
- **Different CCR** for each product type (Loan 100%, Guarantee 50%, L/C 20%)
- **Shared collateral** - Collateral shared across limits
- **Impact analysis** when outstanding balances change

---

Notebook này minh họa cách tính toán hạn mức khả dụng cho tín dụng ngân hàng với:
- **Hạn mức phân cấp** (tổng/con/cháu)
- **CCR khác nhau** cho từng loại sản phẩm (Vay 100%, Bảo lãnh 50%, L/C 20%)
- **TSĐB liên thông** - Tài sản đảm bảo chia sẻ giữa các hạn mức
- **Phân tích tác động** khi dư nợ thay đổi

## 1. Import Libraries / Import Thư Viện

Import necessary modules for limit calculation / Import các module cần thiết để tính toán hạn mức

In [ ]:
from decimal import Decimal
from limit_calculator import (
    LimitCalculator,
    LimitInfo,
    CollateralInfo,
    print_limit_result,
    format_currency
)

print("✓ Import thành công!")

## 2. Simple Example - Non-Hierarchical Loan Limit / Ví Dụ Đơn Giản - Hạn Mức Vay Không Phân Cấp

Calculate available limit for a single loan limit.

**Data:**
- Approved limit: **80 billion VND**
- Current outstanding: **30 billion VND**
- Collateral: **50 billion VND**
- Unsecured ratio: **15%**
- Max unsecured limit: **20 billion VND**
- CCR: **100%** (working capital loan)

---

Tính hạn mức khả dụng cho một hạn mức vay đơn lẻ.

**Dữ liệu:**
- Hạn mức được cấp: **80 tỷ VNĐ**
- Dư nợ hiện tại: **30 tỷ VNĐ**
- TSĐB: **50 tỷ VNĐ**
- Tỷ lệ tín chấp: **15%**
- Hạn mức tín chấp tối đa: **20 tỷ VNĐ**
- CCR: **100%** (vay vốn lưu động)

In [ ]:
# Collateral information / Thông tin TSĐB
collateral_info = CollateralInfo(
    total_collateral=Decimal('50000000000'),  # 50 billion / 50 tỷ
    unsecured_ratio=Decimal('0.15'),  # 15%
    max_unsecured=Decimal('20000000000')  # 20 billion / 20 tỷ
)

# Initialize calculator / Khởi tạo calculator
calculator = LimitCalculator(collateral_info)

# Loan limit / Hạn mức vay
loan_limit = LimitInfo(
    limit_id='LOAN_001',
    limit_name='Hạn mức vay vốn lưu động',
    approved_limit=Decimal('80000000000'),  # 80 billion / 80 tỷ
    outstanding_amount=Decimal('30000000000'),  # 30 billion / 30 tỷ
    ccr=Decimal('1.0')  # 100%
)

# Calculate / Tính toán
result = calculator.calculate_single_limit(loan_limit)

# Print result / In kết quả
print_limit_result(result)

### Result Analysis / Phân Tích Kết Quả

From the results above, we can see:
- **Available limit**: Around **35-40 billion VND**
- Calculated from approved limit (80 billion) minus outstanding (30 billion)
- Collateral and unsecured credit are used to secure the limit

---

Từ kết quả trên, ta có thể thấy:
- **Hạn mức khả dụng**: Khoảng **35-40 tỷ VNĐ**
- Được tính từ hạn mức được cấp (80 tỷ) trừ đi dư nợ (30 tỷ)
- TSĐB và tín chấp được sử dụng để đảm bảo cho hạn mức

## 3. Complex Example - Hierarchical Limits with Shared Collateral / Ví Dụ Phức Tạp - Hạn Mức Phân Cấp với TSĐB Liên Thông

Calculate available limits for a hierarchical limit system with:
- 1 master limit
- 3 child limits (Loan, Guarantee, L/C)
- Different CCR for each type
- Shared collateral among child limits

**Data:**
- **Master limit**: 100 billion VND
- **Collateral**: 80 billion VND (shared for all)
- **Unsecured ratio**: 20%
- **Max unsecured limit**: 30 billion VND

**Child limits:**
1. Working capital loan: 60 billion, CCR 100%, Outstanding 40 billion
2. Guarantee: 30 billion, CCR 50%, Outstanding 20 billion
3. Sight L/C: 20 billion, CCR 20%, Outstanding 10 billion

---

Tính hạn mức khả dụng cho hệ thống hạn mức phân cấp với:
- 1 hạn mức tổng
- 3 hạn mức con (Vay, Bảo lãnh, L/C)
- CCR khác nhau cho mỗi loại
- TSĐB được chia sẻ giữa các hạn mức con

**Dữ liệu:**
- **Hạn mức tổng**: 100 tỷ VNĐ
- **TSĐB**: 80 tỷ VNĐ (chia sẻ cho tất cả)
- **Tỷ lệ tín chấp**: 20%
- **Hạn mức tín chấp tối đa**: 30 tỷ VNĐ

**Các hạn mức con:**
1. Vay vốn lưu động: 60 tỷ, CCR 100%, Dư nợ 40 tỷ
2. Bảo lãnh: 30 tỷ, CCR 50%, Dư nợ 20 tỷ
3. L/C trả ngay: 20 tỷ, CCR 20%, Dư nợ 10 tỷ

In [ ]:
# Set up collateral information / Thiết lập thông tin TSĐB
collateral_info = CollateralInfo(
    total_collateral=Decimal('80000000000'),  # 80 billion / 80 tỷ
    unsecured_ratio=Decimal('0.20'),  # 20%
    max_unsecured=Decimal('30000000000')  # 30 billion / 30 tỷ
)

calculator = LimitCalculator(collateral_info)

# Master limit / Hạn mức tổng
master_limit = LimitInfo(
    limit_id='MASTER_001',
    limit_name='Hạn mức tổng',
    approved_limit=Decimal('100000000000'),  # 100 billion / 100 tỷ
    outstanding_amount=Decimal('0'),  # Not directly counted / Không tính trực tiếp
    ccr=Decimal('1.0')
)

# Child limits / Các hạn mức con
child_limits = [
    LimitInfo(
        limit_id='LOAN_001',
        limit_name='Vay vốn lưu động',
        approved_limit=Decimal('60000000000'),  # 60 billion / 60 tỷ
        outstanding_amount=Decimal('40000000000'),  # 40 billion / 40 tỷ
        ccr=Decimal('1.0'),  # 100%
        parent_limit_id='MASTER_001'
    ),
    LimitInfo(
        limit_id='GUARANTEE_001',
        limit_name='Bảo lãnh',
        approved_limit=Decimal('30000000000'),  # 30 billion / 30 tỷ
        outstanding_amount=Decimal('20000000000'),  # 20 billion / 20 tỷ
        ccr=Decimal('0.5'),  # 50%
        parent_limit_id='MASTER_001'
    ),
    LimitInfo(
        limit_id='LC_001',
        limit_name='L/C trả ngay',
        approved_limit=Decimal('20000000000'),  # 20 billion / 20 tỷ
        outstanding_amount=Decimal('10000000000'),  # 10 billion / 10 tỷ
        ccr=Decimal('0.2'),  # 20%
        parent_limit_id='MASTER_001'
    )
]

print("✓ Hierarchical limit data set up / Đã thiết lập dữ liệu hạn mức phân cấp")

### Calculate with Collateral Allocation by CCR (Pari-passu) / Tính Toán với Phương Pháp Phân Bổ TSĐB Theo CCR (Pari-passu)

In [ ]:
# Calculate with CCR-based collateral allocation method / Tính toán với phương pháp phân bổ TSĐB theo CCR
master_result, child_results = calculator.calculate_hierarchical_limits(
    master_limit=master_limit,
    child_limits=child_limits,
    allocation_method='ccr'
)

# Print master limit result / In kết quả hạn mức tổng
print("="*70)
print("MASTER LIMIT RESULT / KẾT QUẢ HẠN MỨC TỔNG")
print("="*70)
print_limit_result(master_result)

### Collateral Allocation for Child Limits / Phân Bổ TSĐB Cho Các Hạn Mức Con

Collateral is allocated proportionally to weighted outstanding (Outstanding × CCR) / TSĐB được phân bổ theo tỷ lệ dư nợ quy đổi (Outstanding × CCR)

In [ ]:
print("COLLATERAL ALLOCATION FOR CHILD LIMITS / PHÂN BỔ TSĐB CHO CÁC HẠN MỨC CON:")
print("-" * 70)

# Calculate total weighted outstanding / Tính tổng dư nợ quy đổi
total_weighted = Decimal('0')
for result in child_results:
    weighted = result.outstanding_nominal * result.ccr
    total_weighted += weighted
    print(f"{result.limit_name:20} - Weighted outstanding / Dư nợ quy đổi: {format_currency(weighted)}")

print(f"{'Total weighted outstanding / Tổng dư nợ quy đổi':40} - {format_currency(total_weighted)}")
print()

# Print collateral allocation ratio / In tỷ lệ phân bổ TSĐB
for result in child_results:
    ratio = (result.outstanding_nominal * result.ccr) / total_weighted * 100
    print(f"{result.limit_name:20} - Allocated collateral / TSĐB phân bổ: {format_currency(result.collateral_allocated)} ({ratio:.2f}%)")
print("-" * 70)

### Child Limit Details / Chi Tiết Từng Hạn Mức Con

In [ ]:
# Print child limit details / In chi tiết các hạn mức con
for result in child_results:
    print_limit_result(result, indent=2)

### Save Results for Comparison / Lưu Kết Quả Để So Sánh

Save current results for comparison when changes occur / Lưu lại kết quả hiện tại để so sánh khi có thay đổi

In [ ]:
# Save old results / Lưu kết quả cũ
old_results = {result.limit_id: result for result in child_results}

print("✓ Initial results saved for comparison / Đã lưu kết quả ban đầu để so sánh")

## 4. Impact Analysis When Increasing Loan / Phân Tích Tác Động Khi Vay Thêm

Suppose the customer **borrows an additional 15 billion VND**, increasing the loan outstanding from 40 billion to 55 billion.

**Questions:**
- How does the available limit change?
- How is the collateral allocation for other limits affected?
- Is there still enough collateral for Guarantee and L/C?

---

Giả sử khách hàng **vay thêm 15 tỷ VNĐ**, dư nợ vay tăng từ 40 tỷ lên 55 tỷ.

**Câu hỏi:**
- Hạn mức khả dụng thay đổi như thế nào?
- TSĐB phân bổ cho các hạn mức khác bị ảnh hưởng ra sao?
- Có còn đủ TSĐB cho Bảo lãnh và L/C không?

In [ ]:
# Update loan outstanding - increase by 15 billion / Cập nhật dư nợ vay - tăng 15 tỷ
child_limits_new = [
    LimitInfo(
        limit_id='LOAN_001',
        limit_name='Vay vốn lưu động',
        approved_limit=Decimal('60000000000'),
        outstanding_amount=Decimal('55000000000'),  # 55 billion (increased by 15 billion) / 55 tỷ (tăng 15 tỷ)
        ccr=Decimal('1.0'),
        parent_limit_id='MASTER_001'
    ),
    LimitInfo(
        limit_id='GUARANTEE_001',
        limit_name='Bảo lãnh',
        approved_limit=Decimal('30000000000'),
        outstanding_amount=Decimal('20000000000'),  # Unchanged / Không đổi
        ccr=Decimal('0.5'),
        parent_limit_id='MASTER_001'
    ),
    LimitInfo(
        limit_id='LC_001',
        limit_name='L/C trả ngay',
        approved_limit=Decimal('20000000000'),
        outstanding_amount=Decimal('10000000000'),  # Unchanged / Không đổi
        ccr=Decimal('0.2'),
        parent_limit_id='MASTER_001'
    )
]

# Recalculate / Tính toán lại
master_result_new, child_results_new = calculator.calculate_hierarchical_limits(
    master_limit=master_limit,
    child_limits=child_limits_new,
    allocation_method='ccr'
)

print("="*70)
print("RESULT AFTER BORROWING ADDITIONAL 15 BILLION / KẾT QUẢ SAU KHI VAY THÊM 15 TỶ")
print("="*70)
print_limit_result(master_result_new)

### Compare Collateral Allocation Before and After / So Sánh Phân Bổ TSĐB Trước và Sau

In [ ]:
print("COLLATERAL ALLOCATION COMPARISON / SO SÁNH PHÂN BỔ TSĐB:")
print("="*80)
print(f"{'Limit / Hạn mức':<20} | {'Old / Cũ (bil)':>15} | {'New / Mới (bil)':>15} | {'Change / Thay đổi (bil)':>20}")
print("-"*80)

for result_new in child_results_new:
    result_old = old_results[result_new.limit_id]
    
    old_val = result_old.collateral_allocated
    new_val = result_new.collateral_allocated
    change = new_val - old_val
    sign = '+' if change >= 0 else ''
    
    print(f"{result_new.limit_name:20} | {float(old_val)/1e9:>15.2f} | {float(new_val)/1e9:>15.2f} | {sign}{float(change)/1e9:>20.2f}")

print("-"*80)

### Compare Available Limits / So Sánh Hạn Mức Khả Dụng

In [ ]:
print("\nAVAILABLE LIMIT COMPARISON / SO SÁNH HẠN MỨC KHẢ DỤNG:")
print("="*80)
print(f"{'Limit / Hạn mức':<20} | {'Old / Cũ (bil)':>12} | {'New / Mới (bil)':>12} | {'Change / Thay đổi (bil)':>20}")
print("-"*80)

for result_new in child_results_new:
    result_old = old_results[result_new.limit_id]
    
    old_avail = result_old.available_limit_nominal
    new_avail = result_new.available_limit_nominal
    change = new_avail - old_avail
    sign = '+' if change >= 0 else ''
    
    print(f"{result_new.limit_name:20} | {float(old_avail)/1e9:>12.2f} | {float(new_avail)/1e9:>12.2f} | {sign}{float(change)/1e9:>20.2f}")

print("-"*80)

### Limit Details After Changes / Chi Tiết Hạn Mức Sau Thay Đổi

In [ ]:
# Print limit details after changes / In chi tiết các hạn mức sau khi thay đổi
for result in child_results_new:
    print_limit_result(result, indent=2)

## 5. Analysis and Conclusion / Phân Tích và Kết Luận

### Key Results:

1. **Impact of shared collateral:**
   - When loan outstanding increases → Allocated collateral for loans increases
   - Remaining collateral for Guarantee and L/C decreases
   - Available limits of other products are affected

2. **Role of CCR:**
   - CCR 100% (Loan): Consumes most collateral
   - CCR 50% (Guarantee): Consumes medium collateral
   - CCR 20% (L/C): Consumes least collateral
   
3. **Risk management:**
   - Need to monitor total weighted outstanding
   - Ensure sufficient collateral for all limits
   - Balance between unsecured and secured credit

### Usage Notes:

- **CCR allocation method (Pari-passu)**: Fair based on risk proportion
- **Priority allocation method**: Prioritizes more important limits
- **Shared collateral**: Changing one limit affects other limits

---

### Kết Quả Quan Trọng:

1. **Tác động của TSĐB liên thông:**
   - Khi dư nợ vay tăng → TSĐB phân bổ cho vay tăng
   - TSĐB còn lại cho Bảo lãnh và L/C giảm
   - Hạn mức khả dụng của các sản phẩm khác bị ảnh hưởng

2. **Vai trò của CCR:**
   - CCR 100% (Vay): Chiếm nhiều TSĐB nhất
   - CCR 50% (Bảo lãnh): Chiếm TSĐB trung bình
   - CCR 20% (L/C): Chiếm TSĐB ít nhất
   
3. **Quản lý rủi ro:**
   - Cần theo dõi tổng dư nợ quy đổi
   - Đảm bảo đủ TSĐB cho tất cả hạn mức
   - Cân nhắc giữa tín chấp và TSĐB

### Lưu Ý Khi Sử Dụng:

- **Phương pháp phân bổ CCR (Pari-passu)**: Công bằng dựa trên tỷ trọng rủi ro
- **Phương pháp phân bổ theo ưu tiên**: Ưu tiên cho hạn mức quan trọng hơn
- **TSĐB liên thông**: Thay đổi một hạn mức ảnh hưởng đến các hạn mức khác

## 6. Free Testing / Thử Nghiệm Tự Do

You can change the parameters below to test different scenarios: / Bạn có thể thay đổi các tham số dưới đây để thử nghiệm các tình huống khác:

In [ ]:
# Test with custom parameters / Thử nghiệm với các tham số tùy chỉnh
custom_collateral_info = CollateralInfo(
    total_collateral=Decimal('100000000000'),  # Change collateral / Thay đổi TSĐB
    unsecured_ratio=Decimal('0.25'),  # Change unsecured ratio / Thay đổi tỷ lệ tín chấp
    max_unsecured=Decimal('50000000000')  # Change max unsecured limit / Thay đổi hạn mức tín chấp tối đa
)

custom_calculator = LimitCalculator(custom_collateral_info)

# Create custom limit / Tạo hạn mức tùy chỉnh
custom_limit = LimitInfo(
    limit_id='CUSTOM_001',
    limit_name='Hạn mức tùy chỉnh',
    approved_limit=Decimal('120000000000'),  # Change limit / Thay đổi hạn mức
    outstanding_amount=Decimal('50000000000'),  # Change outstanding / Thay đổi dư nợ
    ccr=Decimal('0.8')  # Change CCR / Thay đổi CCR
)

# Calculate / Tính toán
custom_result = custom_calculator.calculate_single_limit(custom_limit)
print_limit_result(custom_result)